In [1]:
from dask import dataframe as df
from dask import delayed 
import dask
import nltk
import re,string,unicodedata

#pred prvim zagonom na novem sistemu
#nltk.download('stopwords')

In [2]:
#source https://examples.dask.org/machine-learning/text-vectorization.html
from dask.distributed import Client, progress

client = Client(n_workers=4, threads_per_worker=2, memory_limit='2GB')
client

/opt/conda/lib/python3.9/site-packages/distributed/node.py:180: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 42161 instead
  warnings.warn(


Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:42161/status,
Dashboard: http://127.0.0.1:42161/status,Workers: 4
Total threads: 8,Total memory: 7.45 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:39585,Workers: 4
Dashboard: http://127.0.0.1:42161/status,Total threads: 8
Started: Just now,Total memory: 7.45 GiB
Comm: tcp://127.0.0.1:45785,Total threads: 2
Dashboard: http://127.0.0.1:46087/status,Memory: 1.86 GiB
Nanny: tcp://127.0.0.1:35779,


In [41]:
import dask.dataframe as dd
import pandas as pd

#df = dd.from_pandas(pd.DataFrame({"review": bunch.data, "target": bunch.target}),npartitions=25)
df=dd.from_pandas(pd.read_csv("IMDB Dataset.csv"),npartitions=25)
df

,review,sentiment
npartitions=25,,
0,object,object
2000,...,...
...,...,...
48000,...,...
49999,...,...


In [4]:
client.shutdown()

Preprocesing oz. čiščenje besedila

In [4]:
#Text cleaning
@delayed
def para_cleaning(x):
    x = re.sub('[,\.!?:()"]', '', x)
    x = re.sub('<.*?>', ' ', x)
    x = re.sub('http\S+', ' ', x)
    x = re.sub('[^a-zA-Z0-9]', ' ', x)
    x = re.sub('\s+', ' ', x)
    return x.lower().strip()

# tokennize
@delayed
def tokennize(x):
    words=nltk.tokenize.word_tokenize(x)
    return words
#Odstranjevanj stopwords


@delayed
def para_sw_remove(x):
    words = nltk.tokenize.word_tokenize(x)
    filtered_list = [word for word in words if word not in sw_set]
    return ' '.join(filtered_list)

In [5]:
#Izbira stopwords kot so am, not, a, the ...
sw_set = set(nltk.corpus.stopwords.words('english'))

In [6]:
def clean_text2(df):
    
    df["review"] = df.review.map(lambda review:review.lower()).map(para_cleaning).map(para_sw_remove)
    
    return df

Ocene pred čiščenjem besedila

In [7]:
df.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [42]:
%time df=df.map_partitions(clean_text2,meta=df.compute()).compute()

CPU times: user 4.28 s, sys: 432 ms, total: 4.71 s
Wall time: 6.32 s


In [43]:
%time df['review']= dask.compute(*df["review"])

CPU times: user 12min 13s, sys: 13.1 s, total: 12min 26s
Wall time: 12min 20s


Ocene po čiščenju besedila

In [18]:
df.head()

,review,sentiment
0,one reviewers mentioned watching 1 oz episode ...,positive
1,wonderful little production filming technique ...,positive
2,thought wonderful way spend time hot summer we...,positive
3,basically family little boy jake thinks zombie...,negative
4,petter mattei love time money visually stunnin...,positive


In [11]:
print(df.head().loc[0, 'review'][:500])

Delayed('getitem-77de60bf2b8ec41e37e2b2757525289f')


In [20]:
#!pip install dask_ml

In [19]:
import dask_ml.feature_extraction.text

vect = dask_ml.feature_extraction.text.HashingVectorizer()
X = vect.fit_transform(df['review'])
X

<50000x1048576 sparse matrix of type '<class 'numpy.float64'>'
	with 4880750 stored elements in Compressed Sparse Row format>

In [20]:
import numpy as np
from dask_ml import preprocessing

le = preprocessing.LabelEncoder()

y=sentiment_data=le.fit_transform(df['sentiment'])
#y

In [21]:
import numpy as np
import sklearn.linear_model
import sklearn.pipeline

import dask_ml.wrappers

In [22]:
sgd = sklearn.linear_model.SGDClassifier(
    tol=1e-3
)
clf = dask_ml.wrappers.Incremental(
    sgd, scoring='accuracy', assume_equal_chunks=True
)
pipe = sklearn.pipeline.make_pipeline(vect, clf)

In [23]:
%%time 
pipe.fit(df['review'], y,
         incremental__classes=[0, 1]);

CPU times: user 2.72 s, sys: 112 ms, total: 2.83 s
Wall time: 2.71 s


Pipeline(steps=[('hashingvectorizer', HashingVectorizer()),
                ('incremental',
                 Incremental(estimator=SGDClassifier(), scoring='accuracy'))])

In [24]:
%%time
predictions = pipe.predict(df['review'])
predictions

CPU times: user 2.68 s, sys: 48.5 ms, total: 2.73 s
Wall time: 2.64 s


array([1, 1, 1, ..., 1, 0, 0])

In [25]:
%%time
accuracy_score=dask_ml.metrics.accuracy_score(y, predictions)
accuracy_score

CPU times: user 58.3 ms, sys: 8.15 ms, total: 66.4 ms
Wall time: 140 ms


0.89474

In [37]:
import random
import pandas as pd
df_original = pd.read_csv('IMDB Dataset.csv')
test_rev=df['review']
# Izbiranje naključnega reviewa

idx_test = random.randint(0, len(y)-1)
idx_original = test_rev.index[idx_test]
(actual_rev, actual_sent) = df_original.iloc[idx_original]

# Primerjanje original ocene z "naučeno" oceno naključno izbranega 
prediction = predictions[idx_test]
prediction=prediction*accuracy_score if prediction<0.5 else prediction*accuracy_score

prediction_sent = 'positive' if prediction >= 0.5 else 'negative'
prediction=accuracy_score
probability = round(accuracy_score if prediction >= 0.5 else 1-prediction, 2)

# Izpis

print('\033[1m' + 'Review #%d:' % idx_original + '\033[0m' + '\n', actual_rev, '\n')
print('\033[1m' + 'Napovedana ocena:' + '\033[0m', prediction_sent, '(z verjetnostjo %.2f)' % probability, '\n')
print('\033[1m' + 'Dejanska ocena:' + '\033[0m', actual_sent)

Review #36450:
 Alain Delon visits swift, sure vengeance on the ruthless crime family that employed him as a hit-man in the Duccio Tessari thriller "Big Guns" after they accidentally murder his wife and child. Tessari and scenarists Roberto Gandus, Ugo Liberatore of "A Minute to Pray, a Second to Die," and Franco Verucci of "Ring of Death" take this actioneer about a career gunman for the mob right down to the wire. Indeed, "Big Guns" is rather predictable, but it still qualifies as solid entertainment with lots of savage and often sudden killings. Alain Delon of "The Godson" is appropriately laconic as he methodically deals out death to the heads of the mob families who refused to let him retire so that he could enjoy life with his young son and daughter. Richard Conte of "The Godfather" plays a Sicilian crime boss who wants to bury the hatchet with the Delon character, but the rest of his hard-nosed associates want the hit-man dead. Like most crime thrillers in the 1960s and 1970s, "